In [9]:
# from Recommenders.Hybrid.SimilarityMergedHybridRecommender import SimilarityMergedHybridRecommender
# from src.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaCBFRecommender import RP3betaCBFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDItemRecommender
from Recommenders.Hybrid.GeneralizedMergedHybridRecommender import (
    GeneralizedMergedHybridRecommender,
)
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_ElasticNet
from Recommenders.Implicit.FeatureCombinedImplicitALSRecommender import (
    FeatureCombinedImplicitALSRecommender,
)
from Recommenders.Hybrid.GeneralizedSimilarityMergedHybridRecommender import (
    GeneralizedSimilarityMergedHybridRecommender,
)
from Utils.ICM_preprocessing import *
from Utils.data_loader import load_URM, load_ICM
from Utils.confidence_scaling import *
from Utils.write_submission import write_submission
from Data_manager.split_functions.split_train_validation_random_holdout import (
    split_train_in_two_percentage_global_sample,
)
from Evaluation.K_Fold_Evaluator import K_Fold_Evaluator_MAP
from bayes_opt import BayesianOptimization

from Evaluation.Evaluator import EvaluatorHoldout
from tqdm import tqdm
%env OPENBLAS_NUM_THREADS=1


env: OPENBLAS_NUM_THREADS=1


In [2]:
URM_all = load_URM("input/data_train.csv")
ICM_channel = load_ICM("input/data_ICM_channel.csv")
ICM_subgenre = load_ICM("input/data_ICM_subgenre.csv")
ICM_genre = load_ICM("input/data_ICM_genre.csv")
ICM_event = load_ICM("input/data_ICM_event.csv")

ICM_combined_channel = combine(ICM=ICM_channel, URM = URM_all)
ICM_combined_genre = combine(ICM=ICM_genre, URM = URM_all)
ICM_combined_subgenre = combine(ICM=ICM_subgenre, URM = URM_all)
ICM_combined_event = combine(ICM=ICM_event, URM = URM_all)


In [34]:
print(URM_all.shape, ICM_channel.shape, ICM_subgenre.shape, ICM_genre.shape, ICM_event.shape)

(13650, 18059) (18059, 213) (18059, 113) (18059, 8) (18059, 358070)


In [3]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(
    URM_all, train_percentage=0.80
)


In [4]:
# evaluator_validation = K_Fold_Evaluator_MAP(
#     URM_validation, cutoff_list=[10], verbose=False
# )

#ICM_combined = combine(ICM=ICM_all, URM=URM_train)


In [5]:
# IALS_recommender_channel = FeatureCombinedImplicitALSRecommender(
#     URM_train=URM_train, ICM_train=ICM_channel, verbose=True
#     )

# IALS_recommender_channel.fit(
#     factors=int(398.601583855084),
#     regularization=0.01,
#     use_gpu=False,
#     iterations=int(94.22855449116447),
#     num_threads=6,
#     confidence_scaling=linear_scaling_confidence,
#     **{
#         "URM": {"alpha": 42.07374324671451}, 
#         "ICM": {"alpha": 41.72067133975204}}
# )

# #-----------------------------------------
# IALS_recommender_subgenre = FeatureCombinedImplicitALSRecommender(
#     URM_train=URM_train, ICM_train=ICM_subgenre, verbose=True
#     )

# IALS_recommender_subgenre.fit(
#     factors=int(398.601583855084),
#     regularization=0.01,
#     use_gpu=False,
#     iterations=int(94.22855449116447),
#     num_threads=6,
#     confidence_scaling=linear_scaling_confidence,
#     **{
#         "URM": {"alpha": 42.07374324671451}, 
#         "ICM": {"alpha": 41.72067133975204}}
# )

# #-----------------------------------------
# IALS_recommender_genre = FeatureCombinedImplicitALSRecommender(
#     URM_train=URM_train, ICM_train=ICM_genre, verbose=True
#     )

# IALS_recommender_genre.fit(
#     factors=int(398.601583855084),
#     regularization=0.01,
#     use_gpu=False,
#     iterations=int(94.22855449116447),
#     num_threads=6,
#     confidence_scaling=linear_scaling_confidence,
#     **{
#         "URM": {"alpha": 42.07374324671451}, 
#         "ICM": {"alpha": 41.72067133975204}}
# )
# #-------------------------------------------
# IALS_recommender_event = FeatureCombinedImplicitALSRecommender(
#     URM_train=URM_train, ICM_train=ICM_event, verbose=True
#     )

# IALS_recommender_event.fit(
#     factors=int(398.601583855084),
#     regularization=0.01,
#     use_gpu=False,
#     iterations=int(94.22855449116447),
#     num_threads=6,
#     confidence_scaling=linear_scaling_confidence,
#     **{
#         "URM": {"alpha": 42.07374324671451}, 
#         "ICM": {"alpha": 41.72067133975204}}
# )


In [6]:
rp3betaCBF_recommender_subgenre =  RP3betaCBFRecommender(
        URM_train=URM_train, 
        ICM_train=ICM_combined_subgenre, 
        verbose=False
)

#-------------------------
rp3betaCBF_recommender_genre =  RP3betaCBFRecommender(
        URM_train=URM_train, 
        ICM_train=ICM_combined_genre, 
        verbose=False
)
#-------------------------
rp3betaCBF_recommender_channel=  RP3betaCBFRecommender(
        URM_train=URM_train, 
        ICM_train=ICM_combined_channel, 
        verbose=False
)
#-------------------------
rp3betaCBF_recommender_event =  RP3betaCBFRecommender(
        URM_train=URM_train, 
        ICM_train=ICM_combined_event, 
        verbose=False
)

rp3beta_recommenders = [rp3betaCBF_recommender_subgenre, rp3betaCBF_recommender_genre, rp3betaCBF_recommender_channel, rp3betaCBF_recommender_event]

In [38]:
for rp3 in tqdm(rp3beta_recommenders):
    rp3.fit(
        topK=int(529.1628484087545),
        alpha=0.7,
        beta=0.3,
        implicit=True,
    )


100%|██████████| 4/4 [01:47<00:00, 26.81s/it]


In [39]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13641 ( 0.1%) Users that have less than 1 test interactions


In [40]:
for rp3 in tqdm(rp3beta_recommenders):
    result_df, _ = evaluator_validation.evaluateRecommender(rp3)
    print(result_df.loc[10]["MAP"])

  0%|          | 0/4 [00:00<?, ?it/s]

EvaluatorHoldout: Processed 13641 (100.0%) in 6.99 sec. Users per second: 1953


 25%|██▌       | 1/4 [00:06<00:20,  6.99s/it]

0.20912788011864294
EvaluatorHoldout: Processed 13641 (100.0%) in 6.90 sec. Users per second: 1977


 50%|█████     | 2/4 [00:13<00:13,  6.94s/it]

0.20904076820230313
EvaluatorHoldout: Processed 13641 (100.0%) in 7.12 sec. Users per second: 1917


 75%|███████▌  | 3/4 [00:21<00:07,  7.02s/it]

0.2091322640778324
EvaluatorHoldout: Processed 13641 (100.0%) in 6.87 sec. Users per second: 1986


100%|██████████| 4/4 [00:27<00:00,  6.97s/it]

0.20844995188408524


In [41]:
#write_submission(recommender=recommender, target_users_path="input/data_target_users_test.csv",
#                    out_path='output/{}_submission.csv'.format(recommender.RECOMMENDER_NAME))

In [42]:
#print('output/{}_submission.csv'.format(recommender.RECOMMENDER_NAME))

In [43]:
# p3alpha_recommender = P3alphaRecommender(
#     URM_train=URM_all,
#     verbose=False
# )
#
# p3alpha_recommender.fit(
#     topK=int(212.8832860130684),
#     alpha=0.4729294763382114,
#     implicit=True
# )


In [44]:
# RP3 BETA (Combined) -----------------------------------------

# rp3betaCombined_recommender= RP3betaCBFRecommender(
#     URM_train=URM_all,
#     ICM_train=ICM_combined,
#     verbose=False
# )

# rp3betaCombined_recommender.fit(
#     topK=int(529.1628484087545),
#     alpha=0.45304737831676245,
#     beta=0.226647894170121,
#     implicit=True
# )


In [45]:
# IALS -----------------------------------------

# IALS_recommender= FeatureCombinedImplicitALSRecommender(
#     URM_train=URM_all,
#     ICM_train=ICM_all,
#     verbose=True
# )

# IALS_recommender.fit(
#     factors=int(398.601583855084),
#     regularization=0.01,
#     use_gpu=False,
#     iterations=int(94.22855449116447),
#     num_threads=6,
#     confidence_scaling=linear_scaling_confidence,
#     **{
#         'URM': {"alpha": 42.07374324671451},
#         'ICM': {"alpha": 41.72067133975204}
#     }
# )


In [46]:
# SLIM -----------------------------------------

# SLIM_recommender = MultiThreadSLIM_ElasticNet(
#         URM_train=ICM_combined.T,
#         verbose=False
#     )

# SLIM_recommender.fit(
#     alpha=0.00026894910579512645,
#     l1_ratio=0.08074126876487486,
#     topK=int(395.376118479588),
#     workers=6
# )

# SLIM_recommender.URM_train = URM_all


In [47]:
# OTHER STUFF ----------------------------------

# rp3betaCBF_recommender= RP3betaCBFRecommender(
#     URM_train=URM_all,
#     ICM_train=ICM_all,
#     verbose=False
# )
#
# rp3betaCBF_recommender.fit(
#     topK=int(117.1),
#     alpha=0.9882,
#     beta=0.7703,
#     implicit=False
# )

# itemKNN_recommender= ItemKNNCFRecommender(
#     URM_train=URM_all,
#     verbose=False
# )
#
# itemKNN_recommender.fit(
#     topK=100,
#     shrink=50
# )
#
# pureSVD_recommender= PureSVDItemRecommender(
#     URM_train=URM_all,
#     verbose=False
# )
#
#
# pureSVD_recommender.fit(
#     num_factors=772,
#     topK= 599
# )


In [48]:
# Generalized Merged Hybrid -----------------------------------------

recommender = GeneralizedMergedHybridRecommender(
    URM_train=URM_train,
    recommenders=[
        rp3betaCBF_recommender_subgenre,
        rp3betaCBF_recommender_channel
    ],
    verbose=False
)


In [49]:
recommender.fit(
    alphas=[
        0.9,
        0.1
    ]
)


In [50]:
result_df, _ = evaluator_validation.evaluateRecommender(recommender)
print(result_df.loc[10]["MAP"])

EvaluatorHoldout: Processed 13641 (100.0%) in 10.27 sec. Users per second: 1329
0.20917159171172794


In [51]:
# write_submission(recommender=recommender, target_users_path="input/data_target_users_test.csv",
#                     out_path='output/{}_submission.csv'.format(recommender.RECOMMENDER_NAME))
